## Sample generation
#### load trained model -> generate samples

In [ ]:
from tensorflow import keras

path = ''
file_name = ''

model = keras.load_model(path + file_name)

### Generate seed

In [ ]:
import numpy as np

INPUT_SIZE = 256
# [batch_size, sequence_len, input_dim]
# generate random seed
seed = np.random.random((1, 1, INPUT_SIZE))

### Generate sequence

In [ ]:
# at default 1 frame = 0.025s
SEQUENCE_LENGTH = 4000
